In [1]:
import pandas as pd
import xgboost as xgb
import operator
from matplotlib import pylab as plt

In [2]:
def ceate_feature_map(features):
    outfile = open('xgb.fmap', 'w')
    i = 0
    for feat in features:
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
        i = i + 1

    outfile.close()

In [3]:
train = pd.read_csv("training.csv")

In [4]:
features = list(train.columns[:-1])
#features

In [5]:
y_train = train.sales
#y_train

In [6]:
def get_data():
    train = pd.read_csv("training.csv")

    features = list(train.columns[:-1])

    y_train = train.sales
    y_train[y_train>0] = 1
    for feat in train.columns[:-1]:
        m = train.groupby([feat])['sales'].mean()
        train[feat].replace(m,inplace=True)

    x_train = train[features]

    return features, x_train, y_train

In [7]:
features, x_train, y_train = get_data()
ceate_feature_map(features)

xgb_params = {"objective": "binary:logistic", "eta": 1, "max_depth": 2, "seed": 42, "silent": 1}
num_rounds = 2

dtrain = xgb.DMatrix(x_train, label=y_train)
gbdt = xgb.train(xgb_params, dtrain, num_rounds)

importance = gbdt.get_fscore(fmap='xgb.fmap')
importance = sorted(importance.items(), key=operator.itemgetter(1))

df = pd.DataFrame(importance, columns=['feature', 'fscore'])
df['fscore'] = df['fscore'] / df['fscore'].sum()

plt.figure()
df.plot()
df.plot(kind='barh', x='feature', y='fscore', legend=False, figsize=(6, 10))
plt.title('XGBoost Feature Importance')
plt.xlabel('relative importance')
plt.gcf().savefig('feature_importance_xgb.png')

/Users/zhengneng/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/zhengneng/anaconda/lib/python2.7/site-packages/matplotlib/axes/_base.py:2562: UserWarning: Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=0.0, right=0.0
  'left=%s, right=%s') % (left, right))
